In [240]:
import pandas as pd
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, Flatten
from keras.callbacks import TensorBoard

print(keras.__version__)

import numpy as np

import gensim
from gensim.models import Word2Vec, KeyedVectors
import gensim.models.keyedvectors as word2vec
from gensim.parsing.preprocessing import strip_non_alphanum, strip_multiple_whitespaces, remove_stopwords, preprocess_string
from gensim.parsing.preprocessing import STOPWORDS

import string
import re

from statistics import mean
from ordered_set import OrderedSet

from openpyxl import load_workbook
print(gensim.__version__)

2.1.3
3.5.0


In [2]:
df = pd.read_csv('pdata.csv')
#model = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, limit=10000)
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)#, limit=10000)
len(model['car']) #300 dimension vectors

300

In [231]:
def get_available_tokens(item,see_vals=False):
    token_list = item.strip('[]').replace(',',' ').replace('/',' ').replace(':',' ').replace(';',' ').split()
    processed_tokens = [x.strip("'").lower() for x in token_list]
    processed_tokens_in_vocab = [x for x in processed_tokens if x in model.vocab]
    translated = re.sub(' +', ' ',item.translate(table)).lower()
    
    translated_in_vocab = [x for x in translated.split() if x in model.vocab and len(x) > 1 and x != 'tru']
    ###print('Translated In Vocab:', translated_in_vocab)
    ###print(len(translated_in_vocab),end='\n\n')
    if not see_vals:
        return len(translated_in_vocab)
    elif see_vals:
        return list(OrderedSet(translated_in_vocab))
def normalize_lengths(token):
    pro_token = []
    if len(token) > avg_len:
        pro_token = token[-avg_len:]
    #if less items, pad with arrays of 0s
    elif len(token) < avg_len:
        pro_token = token + [''] * (avg_len-len(token))
    else:
        pro_token = token
    return pro_token

def create_label_oh(labels):
    unique_labels = list(OrderedSet(labels))
    oh_len = len(unique_labels)
    label_to_oh = {}
    ind_to_label = {}
    for i,lab in enumerate(unique_labels):
        oh = [0] * oh_len
        oh[i] = 1
        label_to_oh[lab] = oh
        ind_to_label[i] = lab
    return label_to_oh, ind_to_label
    
label_oh_dict, ind_to_label_dict = create_label_oh(df['label'])


def make_prediction(norm_token, p=False):
    print(norm_token)
    test = np.array([model[x] if x != '' else np.zeros([300], dtype=float) for x in norm_token])
    test = test.reshape(1, 5, 300)
    pred = nn_model.predict(test)
    
    #print(max(pred[0].tolist()))
    conf = max(pred[0].tolist())
    rconf = round(conf*100, 2)
    label_guess = ind_to_label_dict[pred[0].tolist().index(max(pred[0]))]
    if p:
        print('Confidence: {}%'.format(rconf))
        print(label_guess)
    elif not p:
        return label_guess

def categorize_excel_data(raw_str):
    split = raw_str.split('> ')
    str_split = '","'.join([x.strip() for x in split])
    list_as_str = '["'+str_split+'"]'
    av_tokens = normalize_lengths(get_available_tokens(list_as_str, True))
    pred = make_prediction(av_tokens)
    return pred
tdata = categorize_excel_data('Tru Optik > V12Data > Auto Budget > Auto Budget Range <= $29K')
print(tdata)

#x_oh = np.array([model[x] if x != '' else np.zeros([300], dtype=float) for x in av_tokens])
#x_oh = x_oh.reshape(1, 5, 300)
#nn_model.predict(x_oh)

['auto', 'budget', 'range', '', '']
Appliances


In [232]:
table = str.maketrans(string.punctuation, ' '*len(string.punctuation))#, string.punctuation)

not_in_vocab = []
in_vocab = []
available_lens = []
available_tokens = []
two_lens = []
for item in df['data']:
    available_lens.append(get_available_tokens(item))
    available_tokens.append(get_available_tokens(item, True))
    if get_available_tokens(item) <= 2:
        two_lens.append(get_available_tokens(item, True))

print(available_tokens)
avg_len = int(mean(available_lens))

[['auto', 'budget', 'range'], ['auto', 'budget', 'range'], ['auto', 'budget', 'range'], ['in', 'auto', 'market', 'predictor', 'luxury', 'full', 'size'], ['in', 'auto', 'market', 'predictor', 'mid', 'size', 'car'], ['in', 'auto', 'market', 'predictor', 'small', 'car'], ['income', 'range', '9k'], ['cross', 'pixel', 'financial', 'auto', 'home', 'life', 'insurance'], ['cross', 'pixel', 'financial', 'health', 'insurance'], ['cross', 'pixel', 'real', 'estate', 'home', 'buyers'], ['cross', 'pixel', 'shopping', 'pc', 'laptops', 'tablets'], ['cross', 'pixel', 'travel', 'us', 'canada'], ['cross', 'pixel', 'travel', 'caribbean', 'central', 'south', 'america'], ['cross', 'pixel', 'travel', 'eu', 'africa', 'asia', 'pacific'], ['cross', 'pixel', 'home', 'garden', 'shoppers'], ['media', 'source', 'solutions', 'ethnicity', 'african', 'american'], ['media', 'source', 'solutions', 'ethnicity', 'hispanic'], ['behavioral', 'avid', 'gamer'], ['behavioral', 'sports', 'fanatic'], ['games', 'sports'], ['movie

In [233]:
normalized_tokens = [normalize_lengths(token) for token in available_tokens]
data, labels = [], []
print(normalized_tokens[0])
for item in zip(normalized_tokens, df['label']):
    np_data = [model[x] if x != '' else np.zeros([300], dtype=float) for x in item[0]]
    data.append(np_data)
    labels.append(label_oh_dict[item[1]])

print(np.array(data[0]).shape)
print(np.array(data).shape)

['auto', 'budget', 'range', '', '']
(5, 300)
(1407, 5, 300)


In [244]:
output_len = len(label_oh_dict)

nn_model = Sequential()

nn_model.add(Dense(128, input_shape=(5, 300),activation='relu'))
nn_model.add(Dropout(.25))
nn_model.add(Flatten())
nn_model.add(Dense(64, activation='relu'))
nn_model.add(Dense(output_len, activation='softmax'))


nn_model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Adam(lr=1e-4), 
             metrics=['accuracy'])


In [245]:
data = np.array(data)
labels = np.array(labels)
nn_model.fit(data,labels,
          epochs=100,
          verbose=1,
          shuffle=True,
          validation_split=.15)
    
          

Train on 1195 samples, validate on 212 samples
Epoch 1/100
1195/1195 [==============================] - 1s 736us/step - loss: 3.0108 - acc: 0.3180 - val_loss: 3.0796 - val_acc: 0.1085
Epoch 2/100
1195/1195 [==============================] - 0s 205us/step - loss: 2.5958 - acc: 0.3498 - val_loss: 2.8651 - val_acc: 0.1085
Epoch 3/100
1195/1195 [==============================] - 0s 215us/step - loss: 2.2367 - acc: 0.3715 - val_loss: 2.6330 - val_acc: 0.2075
Epoch 4/100
1195/1195 [==============================] - 0s 207us/step - loss: 2.0271 - acc: 0.4435 - val_loss: 2.3894 - val_acc: 0.3632
Epoch 5/100
1195/1195 [==============================] - 0s 208us/step - loss: 1.8651 - acc: 0.4862 - val_loss: 2.2026 - val_acc: 0.4292
Epoch 6/100
1195/1195 [==============================] - 0s 215us/step - loss: 1.7400 - acc: 0.5272 - val_loss: 2.0618 - val_acc: 0.4528
Epoch 7/100
1195/1195 [==============================] - 0s 221us/step - loss: 1.6336 - acc: 0.5590 - val_loss: 1.9545 - val_acc: 0

1195/1195 [==============================] - 0s 216us/step - loss: 0.2564 - acc: 0.9222 - val_loss: 0.8679 - val_acc: 0.7689
Epoch 61/100
1195/1195 [==============================] - 0s 198us/step - loss: 0.2735 - acc: 0.8971 - val_loss: 0.8677 - val_acc: 0.7689
Epoch 62/100
1195/1195 [==============================] - 0s 195us/step - loss: 0.2554 - acc: 0.9230 - val_loss: 0.8641 - val_acc: 0.7689
Epoch 63/100
1195/1195 [==============================] - 0s 199us/step - loss: 0.2493 - acc: 0.9155 - val_loss: 0.8620 - val_acc: 0.7736
Epoch 64/100
1195/1195 [==============================] - 0s 204us/step - loss: 0.2621 - acc: 0.9146 - val_loss: 0.8558 - val_acc: 0.7736
Epoch 65/100
1195/1195 [==============================] - 0s 197us/step - loss: 0.2410 - acc: 0.9238 - val_loss: 0.8574 - val_acc: 0.7736
Epoch 66/100
1195/1195 [==============================] - 0s 202us/step - loss: 0.2274 - acc: 0.9272 - val_loss: 0.8490 - val_acc: 0.7783
Epoch 67/100
1195/1195 [=======================

In [236]:
print(normalized_tokens[3])
test = np.array([model[x] if x != '' else np.zeros([300], dtype=float) for x in normalized_tokens[0]])

test = test.reshape(1, 5, 300)

pred = nn_model.predict(test)
print(max(pred[0].tolist()))
print(pred[0].tolist().index(max(pred[0])))
print(ind_to_label_dict[pred[0].tolist().index(max(pred[0]))])

['market', 'predictor', 'luxury', 'full', 'size']
0.9750388264656067
0
Auto


In [247]:
#replace the following filepath to use the model - will take the value in column 1 of the file and output the prediction
#onto column 2 - ideally used more as a recommendation engine rather than a fully fleshed out categorizer

wb = load_workbook('/Users/default/Downloads/Final-TruOptik - Monthly Usage - July 2018.xlsx')
ws = wb.active

for i,line in enumerate(ws.iter_rows(max_col=2),start=1):
    row = [x.value for x in line]
    if not row[1]:
        ws.cell(row=i, column=2).value = categorize_excel_data(row[0])
    #print(categorize_excel_data(row[0]))
wb.save('/Users/default/Downloads/Final-TruOptik - Monthly Usage - July 2018.xlsx')
    

In [238]:
model.save('vectorize_v4-model.h5')